## export env
export $(cat .env | xargs)

In [1]:
import os
from pymongo.mongo_client import MongoClient

MONGODB_CONNECTION_STRING = os.environ['MONGODB_CONNECTION_STRING']
MONGODB_DATABASE = os.environ['MONGODB_DATABASE']
MONGODB_RAW_COLLECTION = os.environ['MONGODB_RAW_COLLECTION']

#connect to connection
client = MongoClient(MONGODB_CONNECTION_STRING)
db = client[MONGODB_DATABASE]
collection = db[MONGODB_RAW_COLLECTION]
print("Connecting to MongoDB", MONGODB_DATABASE, MONGODB_RAW_COLLECTION)

#find one document
document = collection.find_one()
print("Found document", document)

Connecting to MongoDB SGPropertyHub raw
Found document {'_id': ObjectId('64f5ede00459b50fd4b6b9af'), 'ad_id': 149578938, 'list_id': 109559324, 'list_time': 1693459941000, 'date': '11 phút trước', 'account_id': 199685, 'account_oid': '0884375cf607675b750f6c59a06da36c', 'account_name': 'huy', 'state': 'accepted', 'subject': 'nhà cống Quỳnh quận 1 dt 55m2 hẻm 1/ giá 7ty8 thương lượng', 'body': 'Nhà đường cống Quỳnh quận 1 dt 3.2*17 55m2 trệt lửng lầu 3ph ngủ nhà hẻm 3m 1/ chủ gấp bán  giải quyết công việc giá chào 7ty8 thương lượng cam kết nhà thật ko kê dưới mọi hình thức xem nhà 24/24', 'category': 1020, 'category_name': 'Nhà ở', 'area': 96, 'area_name': 'Quận 1', 'region': 13, 'region_name': 'Tp Hồ Chí Minh', 'company_ad': True, 'type': 's', 'price': 7800000000, 'price_string': '7,8 tỷ', 'image': 'https://cdn.chotot.com/DIYEG_JbUCQSH-JesmiQa6zCW-ElL7qDEqnfT3WSSwY/preset:listing/plain/1d509b223ab7bbf7d57dbbecc0cc295f-2841153460436950212.jpg', 'webp_image': 'https://cdn.chotot.com/PnzVS3